In [4]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler

#### The data

In [2]:
# train_df = pd.read_csv('../data/train_set_basic.csv')
train_d
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,renal_biopsy_class,anti_cardioliphin_antibodies,anti_b2gp1_antibodies,lupus_anti_coagulant,c3,c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,1,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,2,0,0,0,1,0,0,0,1


In [3]:
def split_x_y(df, target_column):
    '''Splits dataset into features and target and returns them as numpy arrays'''
    y = df[target_column]
    X = df.drop([target_column], axis=1)
    return X, y

In [4]:
def sampling_strategy(X, y, n_samples, t='majority'):
    target_classes = ''
    if t == 'majority': #targeting majority classes
        target_classes = y.value_counts() > n_samples
    elif t == 'minority': #targeting minority classes
        target_classes = y.value_counts() < n_samples
    tc = target_classes[target_classes == True].index
    #target_classes_all = y.value_counts().index
    sampling_strategy_dict = {}
    for target in tc:
        sampling_strategy_dict[target] = n_samples
    return sampling_strategy_dict

In [5]:
def create_balanced_dataset(df, target_column, n_samples, save=False):#DON'T COPY!!!!! IT HAS BEEN TAILORED
    
    X, y = split_x_y(df, target_column)
    under_sampler = RandomUnderSampler(sampling_strategy=sampling_strategy(X, y, n_samples, t='majority'))
    X_under, y_under = under_sampler.fit_resample(X, y)
    
    over_sampler = SMOTE(sampling_strategy=sampling_strategy(X_under, y_under,n_samples, t='minority'),k_neighbors=2)
    X_bal, y_bal = over_sampler.fit_resample(X_under, y_under)
    
    bal_df = pd.DataFrame(X_bal, columns = X.columns)
    
    columns = bal_df.columns
    for col in columns:
        bal_df.loc[bal_df[col] < 0, col] = np.nan
    bal_df['label'] = y_bal
    
    bal_df = bal_df.sample(frac=1).reset_index(drop=True)
        
    if save:
        bal_df.to_csv(f'../data/train_set_basic_balanced.csv', index=False)
    return bal_df


#### end here

In [6]:
balanced_df = create_balanced_dataset(train_df, 'label', 28000, True) #because anemia train dataset has 56k samples

In [7]:
len(balanced_df)

56000